In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
import keras
from keras import backend as K
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [1]:
import torch
torch.cude.is_available()

ModuleNotFoundError: No module named 'torch'

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15213405868177261430
xla_global_id: -1
]


In [ ]:
pd.set_option('display.max_rows', 500)


In [ ]:
test_size = 0.3
val_size = 0.5
train_size =0.7

random_state = 42

In [ ]:
data = pd.read_csv('../feature_sets/post_champ_sel/raw_v1.csv')
data.head()

Calculating VIF values

In [ ]:
# data.to_csv('../feature_sets/post_champ_sel/raw_v1.csv',index=False)

In [ ]:
# X = data[list(data.columns)] 

# # VIF dataframe 
# vif_data = pd.DataFrame() 
# vif_data["feature"] = X.columns 

# vif_data["VIF"] = [variance_inflation_factor(X.values, i) 
#                           for i in range(len(X.columns))] 

# print(vif_data)

In [ ]:
pred = data.pop('blue_team_outcome')

In [ ]:
x_train, x_temp, y_train, y_temp = train_test_split(data,pred,test_size=test_size,random_state = random_state)
x_val, x_test, y_val, y_test = train_test_split(x_temp,y_temp,test_size = val_size,random_state = random_state)

In [ ]:
assert len(data) == len(x_train) + len(x_test) + len(x_val)
assert len(data) == len(y_train) + len(y_val) + len(y_test)

In [ ]:
num_input = len(data.columns)
num_input

In [ ]:
model = Sequential()
model.add(Dense(1024, input_shape=(num_input,), activation='relu'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(8192, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(16384, activation='relu'))
model.add(Dense(16384, activation='leaky_relu'))
model.add(Dropout(0.1))
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='leaky_relu'))
model.add(Dropout(0.1))
model.add(Dense(2048, activation='relu'))
model.add(Dense(2048, activation='leaky_relu'))
model.add(Dropout(0.1))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

In [ ]:
patience = 10
learning_rate = 1e-6
epochs =75
batch_szie=128

In [ ]:
earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, verbose=0, mode='min')
mcp_save = keras.callbacks.ModelCheckpoint('mdl_wts.keras', save_best_only=True, monitor='val_loss', mode='min')
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=learning_rate), metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_szie,verbose=True, validation_data=(x_val, y_val))

In [ ]:
_, accuracy = model.evaluate(x_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))